In [1]:
######只要TTF的時間排序不改就可以匯入所有價格牛逼#######
import json
import xlwings as xw
from datetime import datetime, timedelta

# 讀取 JSON 檔案 #今天的html
json_file_path = r"your json table path"
with open(json_file_path, 'r', encoding='utf-8') as json_file:
    data = json.load(json_file)

# 初始化價格和位置
henry_hub_price = None
columbia_gulf_price = None
texas_gas_zone_price = None

henry_hub_index = None
columbia_gulf_index = None

# 計算 Henry Hub 與 Columbia Gulf Mainline 的距離
min_distance = float('inf')

# 尋找價格
for table in data:
    for key, rows in table.items():
        for index, row in enumerate(rows):
            if isinstance(row, list) and len(row) >= 2:
                # 查找 Columbia Gulf Mainline
                if "Columbia Gulf Mainline" in row:
                    columbia_gulf_price = row[row.index("Columbia Gulf Mainline") + 2]
                    columbia_gulf_index = index  # 記錄 Columbia Gulf Mainline 的行號
                
                # 查找 Henry Hub 價格
                if "Henry Hub" in row:
                    henry_hub_temp_price = row[row.index("Henry Hub") + 2]  # 暫時獲取 Henry Hub 價格
                    henry_hub_temp_index = index  # 暫時記錄 Henry Hub 的行號
                    
                    # 如果已經找到 Columbia Gulf Mainline，則計算與之的距離
                    if columbia_gulf_index is not None:
                        distance = abs(henry_hub_temp_index - columbia_gulf_index)
                        # 如果這個 Henry Hub 比較近，則更新最終的價格和行號
                        if distance < min_distance:
                            min_distance = distance
                            henry_hub_price = henry_hub_temp_price
                            henry_hub_index = henry_hub_temp_index

                # 查找 Texas Gas Zone 1
                if "Texas Gas Zone 1" in row:
                    texas_gas_zone_price = row[row.index("Texas Gas Zone 1") + 2]

# 輸出提取的價格
print(f"Henry Hub Price: {henry_hub_price}")
print(f"Columbia Gulf Mainline Price: {columbia_gulf_price}")
print(f"Texas Gas Zone 1 Price: {texas_gas_zone_price}")

# 加載 Excel 檔案 #上次的價格表 #記得要把公用資料夾裡面的東西全關掉
file_e_path = r'your price excel file'

wb = xw.Book(file_e_path)
ws_prices = wb.sheets['0_Prices']

# 使用 Excel 複製功能複製第五行，並在第五行之後插入一行
ws_prices.api.Rows(5).Copy()  # 複製第五行
ws_prices.api.Rows(5).Insert()  # 插入新行（複製後的新第五行）

# 在新插入的第五行中修改D5的值為昨天的日期
#yesterday = (datetime.today() - timedelta(days=1)).strftime('%Y-%m-%d')
#ws_prices.range('D5').value = yesterday

# 計算「昨日日期」，如果是星期一則設為上週五
today = datetime.today()
if today.weekday() == 0:  # 今天是星期一 (weekday() 回傳 0 表示星期一)
    yesterday = today - timedelta(days=3)  # 上週五
else:
    yesterday = today - timedelta(days=1) # 一般情況下是前一天

# 格式化日期為所需格式
yesterday_str = yesterday.strftime('%Y-%m-%d')

# 插入日期到 Excel
ws_prices.range('D5').value = yesterday_str

# 將提取的價格填寫到對應的單元格
ws_prices.range('K5').value = henry_hub_price  # 填寫 Henry Hub 價格到 K5
ws_prices.range('H5').value = columbia_gulf_price  # 填寫 Columbia Gulf Mainline 價格到 H5
ws_prices.range('E5').value = texas_gas_zone_price  # 填寫 Texas Gas Zone 1 價格到 E5

# 加載甲的資料
xlsm_path = r'your future price path'
wb_oil = xw.Book(xlsm_path)
ws_print = wb_oil.sheets['PRINT']

# 填寫數據
ws_prices.range('X5').value = ws_print.range('C4').value  # 甲 PRINT!C4
ws_prices.range('Q5').value = ws_print.range('C15').value  # 甲 PRINT!C15
ws_prices.range('M5').value = ws_print.range('C13').value  # 甲 PRINT!C13

# 檢查 WEST!CZ5 是否為昨天的日期
ws_west = wb_oil.sheets['WEST']
#if ws_west.range('CZ5').value == yesterday:
ws_prices.range('S5').value = ws_west.range('DA6').value  # 甲 WEST!DA5
ws_prices.range('T5').value = ws_west.range('CR5').value  # 甲 WEST!CR4

# 保存乙的修改
today = datetime.today().strftime('%Y%m%d')
output_e_path = r"your new excel file name".format(today)
wb.save(output_e_path)

# 關閉工作簿
wb.close()
wb_oil.close()


Henry Hub Price: 3.310
Columbia Gulf Mainline Price: 3.160
Texas Gas Zone 1 Price: 3.145


com_error: (-2147352567, '發生例外狀況。', (0, None, None, None, 0, -2147352565), None)